# Using constraints in optimization loop

Sigurd Carlsen, June 2019.

In real world experiments it is not always possible to maintain the same space as was specified when the experiment was created. For example one could run out of a certain material for the experiment.

In these cases costraints come in handy. Instead of redefining our space we just implement constraints on the parameter values that can be used in the experiment. These constraints can be removed or changed as the exeriment changes.

So far we have 3 types of constraints: Single, Inclusive and Exclusive.

A "Single" constraint enforces a specific dimension to take on a specific value.
For example: x should always equal 2.0.
An "Inclusive" constraint forces values to only be the ones included in the constraint.
For example: x can only take on values between 2.0 and 3.0
An "Exclusive" constraint forces values to only be the ones excluded in the constraint.
For example: x can not take on values between 2.0 and 3.0

This notebook is an example on how to use these constraints.
Note that constraints can only be used in conjunction with ask() and tell()

Let's start by importing our packages

In [1]:
from ProcessOptimizer.learning import GaussianProcessRegressor
from ProcessOptimizer import Optimizer
from ProcessOptimizer.space import constraints, Space
from ProcessOptimizer.space.constraints import Single, Inclusive, Exclusive, Constraints
from ProcessOptimizer.space import Integer, Categorical, Real
import numpy as np
np.random.seed(1234)

%matplotlib inline
import matplotlib.pyplot as plt
plt.set_cmap("viridis")

<Figure size 432x288 with 0 Axes>

Then we define our space

In [2]:
space = [
        Real(1, 4),
        Integer(1, 4),
        Categorical(list('abcd')),
    ]

We initialize our optimizer and asks for 5 samples.

In [3]:
opt = Optimizer(space, "GP")
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.2343118064815393, 4, 'a'], [3.5060862359092537, 3, 'd'], [2.1765381816450544, 3, 'd'], [1.6200909740185605, 2, 'a'], [2.8338292820563438, 3, 'b']]


After the optimizer has been initialized we can add constraints to it.
This is done by first creating a list of all constraints we want to pass on to the optimizer.
Let us start out simple by creating a list with only one constraint.
First we try the "Single" type constraint.

In [4]:
constraints_list = [
    Single(0, 2.0, 'real')
]

The first argument to "Single" is the index of the dimension for which the constraint should be applied.
The second argument is the value of the constraint, and the third argument is the type of dimension.

Here we have created a constraint on dimension 0. The type of dimension is set to 'real' as the type of dimension 0 in our space object is of type Real. Because dimension is of type 'real' or value 2.0 is of type float

Now we add the constraints to the optimizer and asks for 5 samples to evaluate.
Note that adding constraints should always be done by calling add_constraints() to ensure that the optimizer internally handles new constraints correctly. 

In [5]:
opt.add_constraints(constraints_list)

We can get the constraints by calling opt.get_constraints()

In [6]:
opt.get_constraints()

Constraints([Single(dimension=0, value=2.0, dimension_type=real)])

In [7]:
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.0, 2, 'b'], [2.0, 3, 'd'], [2.0, 3, 'c'], [2.0, 2, 'd'], [2.0, 4, 'a']]


Note how all of the samples satisfies the constraint: Dimension 0 is alawys 2.0

We can remove the constraints again simply by calling remove_constraints()

In [8]:
opt.remove_constraints()
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.9126986889774467, 3, 'a'], [3.4801562480678307, 1, 'c'], [3.0837177098353616, 4, 'd'], [2.4269767047331183, 2, 'a'], [1.1783819004534621, 3, 'c']]


Now let us try with some Inclusive constraints

In [9]:
constraints_list = [
    Inclusive(1, (2, 3), 'integer'),
    Inclusive(2, ('b', 'c'), 'categorical')  
]

Note how we change the dimension_type arguments to 'integer' and 'categorical' to correspond to the "Real" and "Categorical" dimension types of dimension 1 and 2.
The second argument of Inclusive constraints should be a tuple of length 2 for 'integer' and 'real' dimensions. This tuple defines the lower and upper bound of the constraint.
For categorical dimensions the tuple can be more than 2 in length as it describes all the valid values for the dimension.

In [10]:
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.363537062587538, 2, 'c'], [3.249128233823785, 3, 'b'], [3.8406282458081247, 3, 'b'], [1.2868561708471424, 2, 'c'], [1.3806085643113244, 3, 'b']]


Now only values between and including 2 and 3 are drawn for dimension 1 and only categorical values of 'b' or 'c' are drawn for dimension 2.
Note that calling add_constraints always overwrites all existing constraints in the optimizer

Exlusive constraints are called the same way as Inclusive constraints but instead of enforcing that values drawn lie inside the bounds of the constraints values are insted forced to be outside of the bounds of the constraints

In [11]:
constraints_list = [
    Exclusive(1, (2, 3), 'integer'),
    Exclusive(2, ('b', 'c'), 'categorical')  
]

In [12]:
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.2162050397680901, 1, 'a'], [2.902282535547783, 1, 'a'], [1.2736914023189785, 4, 'd'], [2.372951482077583, 4, 'a'], [3.281742888265041, 1, 'd']]


We can use multiple constraints on the same dimension as long as they do not conflict with eachother. For example it is not possible to put two different "Single" constraints on the same dimension.

Here we want values between 1.5 and 2.0 and values between 3.0 and 3.5 to be excluded:

In [13]:
constraints_list = [
    Exclusive(0, (1.5, 2.0), 'real'),
    Exclusive(0,( 3.0, 3.5), 'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[3.5239419562336094, 2, 'd'], [2.5483114437280747, 2, 'd'], [2.932001497433208, 3, 'b'], [3.7643256994769283, 2, 'b'], [2.2528170279123456, 4, 'b'], [2.691135679117284, 4, 'a'], [3.5458515664358865, 3, 'b'], [2.672750803045366, 1, 'b'], [2.8045217055115668, 1, 'a'], [2.0314171305485944, 4, 'd']]


The same can be done with Inclusive constraints

In [14]:
constraints_list = [
    Inclusive(0, (1.0, 1.1), 'real'),
    Inclusive(0, (2.0, 2.1), 'real'),
    Inclusive(0, (3.0, 3.1), 'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[3.0064198925237204, 2, 'b'], [3.0870201322786457, 3, 'c'], [1.0508417044642835, 1, 'd'], [1.0410007953189553, 2, 'c'], [3.057916422339514, 1, 'c'], [1.0411683655789739, 3, 'a'], [3.0956011638891425, 3, 'd'], [1.0806456879802708, 4, 'd'], [3.0902907304846536, 4, 'b'], [2.0821359110278896, 1, 'a']]


We can even mix Inclusive and Exclusive constraints for the same dimension

In [15]:
constraints_list = [
    Inclusive(0, (2.0, 3.0), 'real'),
    Exclusive(0, (2.3, 2.7), 'real')
]
opt.add_constraints(constraints_list)
next_x = opt.ask(n_points = 10)
print(next_x)

[[2.1054498489470848, 3, 'd'], [2.1323260874509113, 2, 'a'], [2.885422721024095, 2, 'a'], [2.9257910391404076, 1, 'c'], [2.7839368748637963, 3, 'b'], [2.1886958843753916, 3, 'c'], [2.8846662347370495, 1, 'b'], [2.9846535526704123, 1, 'c'], [2.0790978037894874, 3, 'c'], [2.7498715802332088, 1, 'c']]


Whatch out for creating invalid constraint combinations. For example: Inclusive(0,(2.0,3.0),'real') and Exclusive(0,(1.5,3.5),'real') leaves no valid values to be drawn

We can also play around with constrained sampling without adding them to the optimizer by using the Constraints- and the Space class. This can be usefull if we want to quickly test if the constraints makes sense

In [16]:
constraints = Constraints(constraints_list,Space(space))
constraints.rvs(n_samples = 10)

[[2.708435969863238, 2, 'b'],
 [2.129416686716499, 1, 'a'],
 [2.939656970858186, 3, 'a'],
 [2.1481321072310604, 2, 'a'],
 [2.1262148055008154, 2, 'd'],
 [2.7509339508011506, 3, 'b'],
 [2.10083932996696, 3, 'c'],
 [2.2506789339302813, 1, 'c'],
 [2.8776492533940483, 3, 'b'],
 [2.8661773939519177, 4, 'a']]

We can also test if a sample that we are interested in lives up to the costraints

In [17]:
sample_valid = [2.1, 2, 'b']
sample_not_valid = [1, 2, 'b']

In [18]:
constraints.validate_sample(sample_valid)

True

In [19]:
constraints.validate_sample(sample_not_valid)

False